<a href="https://colab.research.google.com/github/diogogc/ACH/blob/master/InstaCaptionCreator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criador de Legenda para o Instagram

Essa aplicação tem por objetivo aprender com suas postagens e gerar insights e ideias para Legendas para novos post.

In [1]:
#@title Instalar Bibliotecas
!pip install -q -U google-generativeai
!pip install -q -U instagrapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:00


In [16]:
#@title Libs e Variáveis
# Libs Utilizadas
from instagrapi import Client
import json
import google.generativeai as genai
from google.colab import userdata
import ipywidgets as widgets

insta_user = userdata.get('insta_user')
insta_password = userdata.get('insta_password')
google_api_key = userdata.get('Gemini_API_KEY')

In [18]:
#@title Obtenção dados do Insta

# Classe Post com dados relevantes da postagem
class Post:
    def __init__(self, legenda, qtd_curtidas, qtd_comentarios):
        self.legenda = legenda
        self.qtd_curtidas = qtd_curtidas
        self.qtd_cometarios = qtd_comentarios

# Inicia capturas de dados no Insta
cl = Client()
cl.login(insta_user, insta_password)

user_id = cl.user_id_from_username(insta_user)


# Retorna uma Lista com as postagens
#@markdown Defina a quantidade de posts que será usada para treinar.

qtd_postagens = 50 #@param {type:"integer"}
medias = cl.user_medias_v1(user_id,qtd_postagens)

#lista para armazenar objetos do tipo Post
postList = []

# Transformando objetos do tipo Media em objetos do tipo Post para simplificar
for aMedia in medias:
    aPost = Post(aMedia.caption_text,aMedia.like_count, aMedia.comment_count)
    postList.append(aPost)

#Json da lista de objetos Post para passar ao Gemini
jsonPostList = json.dumps([aPost.__dict__ for aPost in postList])

print(jsonPostList)

[{"legenda": "Minhas Imagens de ontem :) Podia o foco podia estar melhor, mas foi massa!", "qtd_curtidas": 53, "qtd_cometarios": 7}, {"legenda": "Primeira foto do sol com o filtro no telescopio! Prepara\u00e7\u00e3o para o eclipse do dia 14! Foto ao lado, da nasa", "qtd_curtidas": 33, "qtd_cometarios": 2}, {"legenda": "", "qtd_curtidas": 14, "qtd_cometarios": 0}, {"legenda": "Mais uma para conta!!", "qtd_curtidas": 59, "qtd_cometarios": 6}, {"legenda": "", "qtd_curtidas": 23, "qtd_cometarios": 1}, {"legenda": "Raul sendo premiado pelo desempenho nas competi\u00e7\u00f5es de nata\u00e7\u00e3o do ano! Parab\u00e9ns filho!", "qtd_curtidas": 42, "qtd_cometarios": 2}, {"legenda": "20000 Leagues Under the Seas", "qtd_curtidas": 26, "qtd_cometarios": 2}, {"legenda": "Gerando imagens com o #stablediffusion localmente no meu pc.", "qtd_curtidas": 17, "qtd_cometarios": 1}, {"legenda": "", "qtd_curtidas": 23, "qtd_cometarios": 0}, {"legenda": "Primeira corrida de rua :)", "qtd_curtidas": 20, "qtd

In [49]:
#@title Utilizando o Google Gemini

GOOGLE_API_KEY = userdata.get('Gemini_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel(model_name="gemini-1.0-pro")

promptLearning = "Dado o seguinte json com posts de instagram "+jsonPostList+" crie uma categorizacao dos tipos de postagem, e quero como resposta em json apenas das categorias"

chat = model.start_chat(history=[])

response = chat.send_message(promptLearning)
print(response.text)
responseCleanup = response.text.replace('```json', '')
responseCleanup = responseCleanup.replace('```', '')

aCategList = json.loads(responseCleanup)

print(aCategList)

```json
{
  "categorias": [
    "Astronomia",
    "Fotos Pessoais",
    "Memes",
    "Outros"
  ]
}
```
{'categorias': ['Astronomia', 'Fotos Pessoais', 'Memes', 'Outros']}


In [50]:
#@title Criando Prompt Geração de Legenda
#@markdown Primeiro, defina o tema ou principal conteúdo de sua postagem e depois execute esse código.


tema = 'Impressoras 3d' #@param {type:"string"}
#@markdown Após executar, selecione a categoria no drop down que dará o tom de sua postagem.
@widgets.interact(categoriaPrompt=aCategList['categorias'])

def userPrompt(categoriaPrompt):
  if len(categoriaPrompt):
    promptFinal = "Escreva um post no estilo das legendas apresentadas no json originalmente na categoria "+categoriaPrompt+" com tema "+tema
    return chat.send_message(promptFinal).text
  else :
    return 'Escreva um post de astronomia'

# aResponse = chat.send_message(user_prompt)
# while user_prompt != "fim":
#   aResponse = chat.send_message(user_prompt)
#   print("Resposta: ",aResponse.text, "\n")
#   user_prompt = input("Esperando Prompt: ")

interactive(children=(Dropdown(description='categoriaPrompt', options=('Astronomia', 'Fotos Pessoais', 'Memes'…